In [ ]:
import keras
from keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import datasets, layers, models, losses
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
import datetime

In [ ]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

In [ ]:
splitSize = (0.0, 0.765, 0.9, 1.0)

Testing precision on red channel only

In [ ]:
array = np.vstack(tfds.as_numpy(dataset[0]))
ranges = list([int(array.shape[0] * x) for x in splitSize])

def getDataset(ranges, idx):
    return (
        np.array(list(map(lambda x: x[0][:, :, :], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['cooling_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['pollen_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['wasps_output'], array[ranges[idx]: ranges[idx + 1]])))
    )
train_images, train_labels1,train_labels2,train_labels3, train_labels4= getDataset(ranges, 0)
validation_images, validation_labels1,validation_labels2, validation_labels3, validation_labels4 = getDataset(ranges, 1)
test_images, test_labels1,test_labels2,test_labels3, test_labels4  = getDataset(ranges, 2)

<ipython-input-42-621227d31954>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  array = np.vstack(tfds.as_numpy(dataset[0]))
/usr/local/lib/python3.8/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [ ]:
train_labels = np.dstack((train_labels1, train_labels2, train_labels3,train_labels4)).squeeze()
validation_labels = np.dstack((validation_labels1,validation_labels2, validation_labels3, validation_labels4)).squeeze()
test_labels = np.dstack((test_labels1,test_labels2,test_labels3, test_labels4)).squeeze()
train_labels.shape, train_labels[:10]

((5729, 4), array([[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

In [ ]:
def create_model(model_url, num_classes=10):

  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, 
                                           name='feature_extraction_layer',
                                           input_shape=(300,150,3)) 
  model = tf.keras.Sequential([
    feature_extractor_layer, 
    layers.Dense(num_classes, activation='sigmoid', name='output_layer') 
  ])
  return model

In [ ]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
resnet_model = create_model(resnet_url, num_classes=4)
resnet_model.compile(loss= tf.keras.losses.BinaryCrossentropy(),
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
resnet_history = resnet_model.fit(train_images,train_labels,
                                  epochs=50,
                                  batch_size= 32,
                                  validation_data=(validation_images, validation_labels),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub", 
                                                                         experiment_name="resnet50V2")])

Saving TensorBoard log files to: tensorflow_hub/resnet50V2/20221222-080158
Epoch 1/50
180/180 [==============================] - 65s 99ms/step - loss: 8.5964 - accuracy: 0.3051 - val_loss: 3.7701 - val_accuracy: 0.1868
Epoch 2/50
180/180 [==============================] - 17s 95ms/step - loss: 1.6448 - accuracy: 0.3727 - val_loss: 1.4865 - val_accuracy: 0.3053
Epoch 3/50
180/180 [==============================] - 17s 95ms/step - loss: 1.2627 - accuracy: 0.4064 - val_loss: 1.3722 - val_accuracy: 0.3538
Epoch 4/50
180/180 [==============================] - 17s 96ms/step - loss: 1.1526 - accuracy: 0.4338 - val_loss: 2.9617 - val_accuracy: 0.2935
Epoch 5/50
180/180 [==============================] - 17s 96ms/step - loss: 1.4510 - accuracy: 0.4493 - val_loss: 2.3227 - val_accuracy: 0.5128
Epoch 6/50
180/180 [==============================] - 17s 96ms/step - loss: 1.3727 - accuracy: 0.4441 - val_loss: 1.3101 - val_accuracy: 0.5484
Epoch 7/50
180/180 [==============================] - 17s 97m

In [ ]:
def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "ResNet50V2" \
  --description "resnet_log" \
  --one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./tensorflow_hub/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=7gk2wTtTZSzA7SnChellDP8mF3Su7h&prompt=consent&access_type=off

In [ ]:
!tensorboard dev list


https://tensorboard.dev/experiment/i7iXnKWDSkClXmmsPouPcg/
	Name                 ResNet50V2
	Description          resnet_log
	Id                   i7iXnKWDSkClXmmsPouPcg
	Created              2022-12-22 08:19:04 (12 seconds ago)
	Updated              2022-12-22 08:19:11 (5 seconds ago)
	Runs                 6
	Tags                 5
	Scalars              446
	Tensor bytes         0
	Binary object bytes  5551731
Total: 1 experiment(s)
